In [239]:
filename_pickle = "Biomolecules.pkl"
collection_name='NCERT_Biology'
host_name = '20.242.176.11'
model_name = 'all-MiniLM-L6-v2'

In [240]:
import pandas as pd
import pickle

In [241]:
from sentence_transformers import SentenceTransformer

In [242]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from qdrant_client.models import PointStruct

In [243]:
model = SentenceTransformer(model_name)

In [244]:
qdrant_client = QdrantClient(host=host_name, port=6333)

In [245]:
def get_encodings():
    if (filename_pickle != ""):
        with open(filename_pickle, 'rb') as f: 
            df_embeddings = pickle.load(f)
    embeddings_all = df_embeddings["embeddings"].tolist()
    Lines = df_embeddings["text"].tolist()
    return embeddings_all,Lines

In [246]:
embeddings_all,Lines = get_encodings()

In [247]:
total_len = len(Lines)
total_len

19

In [248]:
import uuid

In [249]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(
            id = str(uuid.uuid1()),
            vector=embeddings_all[i],
            payload={"text":Lines[i]}
        )
        for i in range(total_len)
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [268]:
user_input ="What are the keywords for BioMolecules?"

In [269]:
def get_model():    
    model = SentenceTransformer(model_name)
    return model

In [270]:
q_new = user_input
q_new_embeddings = get_model().encode(q_new)

In [271]:
search_result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=q_new_embeddings,
            query_filter=None,  # We don't want any filters for now
            top=3,  # 5 the most closest results is enough
            limit = 3,
            with_vectors=False,  # We don't need vectors in response
        )

In [272]:
len(search_result)

3

In [273]:
assert len(search_result) == 3

In [274]:
search_result

[ScoredPoint(id='f200b689-ef3d-11ed-a788-d41b81ac7ad2', version=1, score=0.5774008, payload={'text': 'material. Polysaccharides are components of cell wall in plants, fungi and also\n of the exoskeleton of arthropods. They also are storage forms of energy (e.g. ,\n starch and glycogen). Proteins serve a variety of cellular functions. Many of\n them are enzymes, some are antibodies, some are receptors, some are hormones\n and some others are structural proteins. Collagen is the most abundant protein\n in animal world and Ribulose bisphosphate Carboxylase-Oxygenase ( RuBisCO )\n is the most abundant protein in the whole of the biosphere.\n Enzymes are proteins which catalyse biochemical reactions in the cells.\n Ribozymes ar e nucleic acids with catalytic power . Proteinaceous enzymes exhibit\n substrate specificity, require optimum temperature and pH for maximal activity.\n They are denatured at high temperatures. Enzymes lower activation energy of\n reactions and enhance greatly the ra

In [275]:
payloads = [hit.payload for hit in search_result]

In [276]:
payloads

[{'text': 'material. Polysaccharides are components of cell wall in plants, fungi and also\n of the exoskeleton of arthropods. They also are storage forms of energy (e.g. ,\n starch and glycogen). Proteins serve a variety of cellular functions. Many of\n them are enzymes, some are antibodies, some are receptors, some are hormones\n and some others are structural proteins. Collagen is the most abundant protein\n in animal world and Ribulose bisphosphate Carboxylase-Oxygenase ( RuBisCO )\n is the most abundant protein in the whole of the biosphere.\n Enzymes are proteins which catalyse biochemical reactions in the cells.\n Ribozymes ar e nucleic acids with catalytic power . Proteinaceous enzymes exhibit\n substrate specificity, require optimum temperature and pH for maximal activity.\n They are denatured at high temperatures. Enzymes lower activation energy of\n reactions and enhance greatly the rate of the reactions. Nucleic acids carry\n hereditary information and are passed on from pa

In [277]:
payloads[0]["text"]

'material. Polysaccharides are components of cell wall in plants, fungi and also\n of the exoskeleton of arthropods. They also are storage forms of energy (e.g. ,\n starch and glycogen). Proteins serve a variety of cellular functions. Many of\n them are enzymes, some are antibodies, some are receptors, some are hormones\n and some others are structural proteins. Collagen is the most abundant protein\n in animal world and Ribulose bisphosphate Carboxylase-Oxygenase ( RuBisCO )\n is the most abundant protein in the whole of the biosphere.\n Enzymes are proteins which catalyse biochemical reactions in the cells.\n Ribozymes ar e nucleic acids with catalytic power . Proteinaceous enzymes exhibit\n substrate specificity, require optimum temperature and pH for maximal activity.\n They are denatured at high temperatures. Enzymes lower activation energy of\n reactions and enhance greatly the rate of the reactions. Nucleic acids carry\n hereditary information and are passed on from parental gen

In [278]:
import openai
key = 'd6bcc40e68fc4119abcd43b4661dc8e3'
location = 'eastus'
endpoint = 'https://openaidemos007.openai.azure.com/'

openai.api_type = "azure"
openai.api_key = key
openai.api_base = endpoint

deployment_id='gta'
deployment_id_gpt4='gpt4'

openai.api_key = key

In [279]:
def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

def generate_answer(prompt):
    response = openai.Completion.create(
    engine=deployment_id,
    prompt=prompt,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response.choices[0].text).strip()

def generate_answer_Chat(conversation):
    openai.api_version = "2023-03-15-preview"
    response = openai.ChatCompletion.create(
    engine=deployment_id_gpt4,
    messages=conversation,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response['choices'][0]['message']['content']).strip()



In [280]:
payloads = [hit.payload for hit in search_result]
metadata = [res["text"] for res in payloads]
df = pd.DataFrame({'content': metadata })
context= "\n\n".join((df["content"]))

In [281]:
context

'material. Polysaccharides are components of cell wall in plants, fungi and also\n of the exoskeleton of arthropods. They also are storage forms of energy (e.g. ,\n starch and glycogen). Proteins serve a variety of cellular functions. Many of\n them are enzymes, some are antibodies, some are receptors, some are hormones\n and some others are structural proteins. Collagen is the most abundant protein\n in animal world and Ribulose bisphosphate Carboxylase-Oxygenase ( RuBisCO )\n is the most abundant protein in the whole of the biosphere.\n Enzymes are proteins which catalyse biochemical reactions in the cells.\n Ribozymes ar e nucleic acids with catalytic power . Proteinaceous enzymes exhibit\n substrate specificity, require optimum temperature and pH for maximal activity.\n They are denatured at high temperatures. Enzymes lower activation energy of\n reactions and enhance greatly the rate of the reactions. Nucleic acids carry\n hereditary information and are passed on from parental gen

In [282]:
prompt = create_prompt(context,q_new)
conversation=[{"role": "system", "content": "You are a helpful assistant."}]
conversation.append({"role": "assistant", "content": prompt})
conversation.append({"role": "user", "content": user_input})
reply = generate_answer_Chat(conversation)

In [283]:
reply

'Some keywords for biomolecules include:\n\n1. Macromolecules\n2. Proteins\n3. Nucleic acids\n4. Polysaccharides\n5. Lipids\n6. Amino acids\n7. Carbohydrates\n8. Enzymes\n9. DNA (Deoxyribonucleic acid)\n10. RNA (Ribonucleic acid)\n11. Monosaccharides\n12. Disaccharides\n13. Fatty acids\n14. Glycerol\n15. Triglycerides\n16. Peptide bonds\n17. Primary structure\n18. Secondary structure\n19. Tertiary structure\n20. Quaternary structure\n21. Primary metabolites\n22. Secondary metabolites\n23. Hormones\n24. Antibodies\n25. Receptors\n26. Collagen\n27. RuBisCO (Ribulose bisphosphate Carboxylase-Oxygenase)\n28. Cellulose\n29. Glycogen\n30. Starch'

In [284]:
result = qdrant_client.get_collection(collection_name=collection_name)

In [285]:
result

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=32, indexed_vectors_count=0, points_count=32, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})